In [2]:
#——————————————————————1.WM,GM,CSF,Whole brain mask construction————————————————————
#input
#ribbon_path. ribbon文件的路径
#wmpacr_path, wmparc文件路径
#fmri_path, 功能像文件路径
#out_Dir, where to save masks

#输出
#本地WM,GN,CSF，whole brain的mask 3D文件
#WM,GN,CSF，whole brain的一维数组，便于后续分析
def maskConstruction (ribbon_path, wmparc_path, fmri_path, out_Dir):
    #——————————读取数据——————————
    ribbon = nib.load(ribbon_path) #读取数据,注意此时仅仅是一个替代（或者说映像proxy），并未将数据加载到内存中
    wmparc = nib.load(wmparc_path)

    #——————————重采样之前，先定义输出文件路径——————————
    #由于重采样之后会重新生成与功能像分辨率一致的结构像文件，因此需要定义输出文件的路径
    ribbon_out = op.join(out_Dir, 'ribbon_out.nii.gz')#输出影像文件地址
    wmparc_out = op.join(out_Dir, 'wmparc_out.nii.gz')

    ribbonMat = op.join(out_Dir, 'ribbon_flirt.mat')#输出矩阵文件地址，这三个矩阵仅仅是函数需要，并没有什么用，事后将移除
    wmparcMat = op.join(out_Dir, 'wmparc_flirt.mat')#输出矩阵文件地址
    eyeMat = op.join(out_Dir, 'eye.mat')#创建一个单位阵进行刚体变换，实际上就是不变换。
                                    #这是由于重采样其实使用了fsl的flirt配准函数，函数需要一个变换矩阵进行配准。但所有图形都已经配准好了，此时仅仅需要重采样
    with open(eyeMat,'w') as fid: #设置为单位矩阵
                fid.write('1 0 0 0\n0 1 0 0\n0 0 1 0\n0 0 0 1')

    #——————————开始重采样——————————
    #调用fsl的FLIRT配准函数进行重采样，由于此时无需配准，因此变换矩阵都是单位阵，仅仅是函数需要
    #重采样方法选用最近邻(nearestneighbour)。这是由于不能改变结构像的数值，因为一旦变化（如取平均），那么便无法与结构的编号进行匹配
    #毕竟wmparc和ribbon里面的数值实际上就是不同组织的编号
    flirt_ribbon = fsl.FLIRT(in_file=ribbon_path, out_file=ribbon_out,#ribbon原始文件以及输出文件的路径
                                reference=fmri_path, apply_xfm=True,#用功能像来作为配准的参照
                                in_matrix_file=eyeMat, out_matrix_file=ribbonMat, interp='nearestneighbour')
    flirt_ribbon.run()
    flirt_wmparc = fsl.FLIRT(in_file=wmparc_path, out_file=wmparc_out,
                                    reference=fmri_path, apply_xfm=True,
                                    in_matrix_file=eyeMat, out_matrix_file=wmparcMat, interp='nearestneighbour')
    flirt_wmparc.run()

    #————————读取重采样后的文件————————
    #加上.get_fdata()后即可读取完整数据，得到一个多维数组
    ribbon_flirt = nib.load(ribbon_out).get_fdata()
    wmparc_flirt = nib.load(wmparc_out).get_fdata()

    #————————创建mask————————
    #根据https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/AnatomicalROI/FreeSurferColorLUT中的索引找出ribbon文件和wmparc文件中对应的灰质/白质/脑脊液位置
            
    # white & CSF matter mask
    # indices are from FreeSurferColorLUT.txt

    # Left-Cerebral-White-Matter, Right-Cerebral-White-Matter
    ribbonWMstructures = [2, 41]#大脑皮层白质，基于ribbon

    # Left-Cerebral-Cortex, Right-Cerebral-Cortex
    ribbonGMstrucures = [3, 42]#大脑皮层灰质，基于ribbon

    #下面的皮层下结构都是基于wmparc
    # Cerebellar-White-Matter-Left, Brain-Stem, Cerebellar-White-Matter-Right
    wmparcWMstructures = [7, 16, 46]#小脑白质，基于wmparc。这里还纳入了脑干，具体原因还需进一步查文献

    # Left-Cerebellar-Cortex, Right-Cerebellar-Cortex, Thalamus-Left, Caudate-Left
    # Putamen-Left, Pallidum-Left, Hippocampus-Left, Amygdala-Left, Accumbens-Left 
    # Diencephalon-Ventral-Left, Thalamus-Right, Caudate-Right, Putamen-Right
    # Pallidum-Right, Hippocampus-Right, Amygdala-Right, Accumbens-Right
    # Diencephalon-Ventral-Right
    wmparcGMstructures = [8, 47, 10, 11, 12, 13, 17, 18, 26, 28, 49, 50, 51, 52, 53, 54, 58, 60]#皮层下灰质

    # Fornix, CC-Posterior, CC-Mid-Posterior, CC-Central, CC-Mid-Anterior, CC-Anterior
    wmparcCCstructures = [250, 251, 252, 253, 254, 255]#胼胝体

    # Left-Lateral-Ventricle, Left-Inf-Lat-Vent, 3rd-Ventricle, 4th-Ventricle, CSF
    # Left-Choroid-Plexus, Right-Lateral-Ventricle, Right-Inf-Lat-Vent, Right-Choroid-Plexus
    wmparcCSFstructures = [4, 5, 14, 15, 24, 31, 43, 44, 63]#脑脊液

    #——————————make masks，将上述索引与重采样后的ribbon和wmparc文件取交集，得到对应组织的位置，用0，1表示————————
    WMmask = np.double(np.logical_and(np.logical_and(np.logical_or(np.logical_or(np.in1d(ribbon_flirt, ribbonWMstructures),
                                                                        np.in1d(wmparc_flirt, wmparcWMstructures)),
                                                            np.in1d(wmparc_flirt, wmparcCCstructures)),
                                            np.logical_not(np.in1d(wmparc_flirt, wmparcCSFstructures))),
                            np.logical_not(np.in1d(wmparc_flirt, wmparcGMstructures))))
    CSFmask = np.double(np.in1d(wmparc_flirt, wmparcCSFstructures))
    GMmask = np.double(np.logical_or(np.in1d(ribbon_flirt,ribbonGMstrucures),np.in1d(wmparc_flirt,wmparcGMstructures)))

    #————————————————————write masks，将mask导出到本地备用——————————————
    ref = nib.load(wmparc_out)#由于上述得到的mask是一维数组，因此需要导入wmparc_flirt作为参照，将矩阵维度还原

    img = nib.Nifti1Image(WMmask.reshape(ref.shape).astype('<f4'), ref.affine)#astype函数用于将数组的数据类型转换为指定的数据类型。'<f4'是一个数据类型描述符，
                                                                            #图像继承参照的affine，是图像空间到世界空间坐标的转换矩阵
    WMmask_outPath = op.join(out_Dir,'WMmask.nii.gz')
    nib.save(img, WMmask_outPath)

    img = nib.Nifti1Image(CSFmask.reshape(ref.shape).astype('<f4'), ref.affine)
    CSFmask_outPath = op.join(out_Dir,'CSFmask.nii.gz')
    nib.save(img, CSFmask_outPath)

    img = nib.Nifti1Image(GMmask.reshape(ref.shape).astype('<f4'), ref.affine)
    GMmask_outPath = op.join(out_Dir,'GMmask.nii.gz')
    nib.save(img, GMmask_outPath)

    # delete temporary files
    cmd = 'rm {} {} {}'.format(eyeMat, ribbonMat, wmparcMat)
    call(cmd,shell=True)

    #——————————读取mask，直接用于后续分析——————————
    tmpWM = nib.load(WMmask_outPath)#读取白质mask
    nRows, nCols, nSlices = tmpWM.header.get_data_shape()#获取数据维度结构
    maskWM = np.asarray(tmpWM.dataobj).reshape(nRows*nCols*nSlices, order='F') > 0#将数据转换为一维数组,用逻辑值表示

    tmpCSF = nib.load(CSFmask_outPath)
    maskCSF = np.asarray(tmpCSF.dataobj).reshape(nRows*nCols*nSlices, order='F')  > 0

    tmpGM = nib.load(GMmask_outPath)
    maskGM = np.asarray(tmpGM.dataobj).reshape(nRows*nCols*nSlices, order='F') > 0

    maskAll_1D  = np.logical_or(np.logical_or(maskWM, maskCSF), maskGM)
    maskWM_1D  = maskWM[maskAll_1D]
    maskCSF_1D = maskCSF[maskAll_1D]
    maskGM_1D  = maskGM[maskAll_1D]

    #——————————输出值——————————
    return maskAll_1D, maskWM_1D, maskCSF_1D, maskGM_1D

In [15]:
#——————————————————————2. deneam normalization————————————————————
#输入
#fmri_path, 功能像文件路径。注意导入的是cifti还是nifti格式
#maskAll, 全脑mask，来自于上一个函数的输出
#iscifti，whether the fmri data is cifti

#输出
#demean_data,标准化后的fmri数据，二维数组， voxel×time
def demean(fmri_path, maskAll, iscifti = True):  
    #——————————读取fMRI数据文件——————————
    img0 = nib.load(fmri_path) #读取影像文件，注意此时数据并没有加载到内存中，仅仅是一个proxy
    img0_data = img0.get_fdata() #读取完整数据，由于数据量很大，耗时较长
    if iscifti:
        img0_masked = img0_data.T #cifti的文件是二维数组，时间×voxel，为了与后面一致，将其转置
        #另外，cifti的文件本身只包含了脑组织，所以并不需要额外的mask
    else:  
        nrow, ncol, nslice, nTR = img0.header.get_data_shape() #获取数据维度结构, 此时是一个四维数据，分别是：行，列，切片，时间点
        #——————————数据维度转换——————————
        img0_2D = img0_data.reshape((nrow * ncol * nslice, nTR),order = 'F')#将四维数据转换成二维，前三个维度合并为一个
                                                                        #order=‘F’表示第一个维度最先展开(nrow)
                                                                        #展开方式也可更换为C，但要保证后续所有的展开方式一致
        img0_2D.shape #此时的img0_2D是二维数据，每一行是一个体素的时间序列
        #——————————利用全脑mask提取fmri脑组织的信号——————————
        img0_masked = img0_2D[maskAll,:] #提取脑组织信号
        
    #——————————去均值处理————————————
    #voxel_means = img0_masked.mean(1)[:,np.newaxis] #为便于后续减去该均值，需要增加一个维度，变成二维数组
    #img_demean = img0_masked - voxel_means
    img_demean= img0_masked - img0_masked.mean()
    return img_demean #返回去均值后的fmri数据

In [4]:
#——————————————————————3.detrending————————————————————
#输入
#demean_fmri, 标准化后的fmri数据，注意不是路径。该数据为上一个函数的输出结果

#输出
#detrending_data,去趋势后的fmri数据，二维数组， voxel×time
def detrending(demean_fmri):
    #构建设计矩阵
    timepoints = np.arange(demean_fmri.shape[1])[:,np.newaxis]#生成二维数据（1200×1），不过只有一列数据，从0到1199的时间序列
    X = np.concatenate((np.ones([demean_fmri.shape[1],1]), timepoints), axis=1)#生成设计矩阵，第一列是截距（全为1），第二列是时间序列
    #回归模型
    fit = np.linalg.lstsq(X, demean_fmri.T, rcond=None)[0]# 获取回归系数,rcond意味着是否设置阈值来去掉较小的奇异值，此处不去掉

    #————————————aggressive regression——————————
    #fittedvalues = np.dot(X, fit)#使用矩阵乘法得到拟合值
    #————————————soft regression——————————
    X1 = X[:,1][:,np.newaxis]
    fit1 = fit[1,:][np.newaxis,:]

    #此外，需要注意的是原始数据每一行才是一个voxel，需要先转置，将每一个voxel变为一列才纳入模型
    #由于现在的data有很多列，因此是每一列单独进行拟合，回归系数也变为一个2（截距，时间）×voxel
    fittedvalues = np.dot(X1, fit1)#使用矩阵乘法得到拟合值

    detrending_data = demean_fmri - fittedvalues.T #原始信号减去拟合值完成去趋势，需要注意的是此时的拟合值需要先转置回来与原始数据结构一致
    return detrending_data

In [5]:
#——————————————————————4.regression denoising————————————————————
#输入
#motionPar_path 头动文件路径
#maskWM，基于nifti数据生成的白质mask
#maskCSF，基于nifti数据生成的脑脊液mask
#maskGM,基于nifti数据生成的灰质mask，最终只分数灰质部分
#dt_Vdata, 去趋势后得到的nifti数据，用于生成信号
#dt_Cdata, 若分析的是cifti文件，那么回归的因变量使用这个去趋势后的数据
#Rmotion,是否要回归掉头动。如果是FIX-ICA数据的话，头动已经回归掉了，此时无需再做一遍
#iscifti，分析的数据是否为cifti格式
def signalRegress(motionPar_path, maskWM, maskCSF, maskGM, dt_Vdata, dt_Cdata, Rmotion=False, RGS = False, iscifti = True):
    motionPar = np.loadtxt(motionPar_path)#读取头动参数
    WM_mean = dt_Vdata[maskWM,:].mean(0)[:,np.newaxis]#白质信号，基于nifti生成
    CSF_mean = dt_Vdata[maskCSF,:].mean(0)[:,np.newaxis]#脑脊液信号，基于nifti生成
    Global_mean = dt_Vdata.mean(0)[:,np.newaxis]#全局信号，基于nifti生成
    X = np.concatenate((np.ones([dt_Vdata.shape[1],1]), WM_mean, CSF_mean), axis=1)
    if Rmotion:
        X = np.concatenate((X, motionPar), axis=1)#加入头动
    if RGS:
        X = np.concatenate((X, Global_mean), axis=1)#加入全局信号
    #——————————————进行回归————————————————
    if iscifti:
        dt_data = dt_Cdata
    else:
        dt_data = dt_Vdata[maskGM,:]
    fit = np.linalg.lstsq(X, dt_data.T, rcond=None)[0]# 获取回归系数,rcond意味着是否设置阈值来去掉较小的奇异值，此处不去掉
    #此外，需要注意的是原始数据每一行才是一个voxel，需要先转置，将每一个voxel变为一列才纳入模型
    #由于现在的data有很多列，因此是每一列单独进行拟合，回归系数也变为一个2（截距，时间）×voxel
    #————————————aggressive regression——————————
    #fittedvalues = np.dot(X, fit)#使用矩阵乘法得到拟合值
    #——————————————————soft regression——————————————
    X1 = X[:,1:]
    fit1 = fit[1:,:]
    
    fittedvalues = np.dot(X1, fit1)#使用矩阵乘法得到拟合值
    denoised_data = dt_data - fittedvalues.T #原始信号减去拟合值完成去组织信号，需要注意的是此时的拟合值需要先转置回来与原始数据结构一致
    return denoised_data

In [6]:
def flitering(data, TR, high=0.01, low=0.08):
    data_flitered = clean(data.T, detrend=False, standardize=False, #用butterworth滤波
                   t_r=TR, high_pass=high, low_pass=low)
    return data_flitered

In [7]:
#——————————————————————导入所需包——————————————————————
import nibabel as nib #影像处理所使用的库
import numpy as np
import os
import os.path as op #操作路径拼接的库
import nipype.interfaces.fsl as fsl #fsl交互的库
from subprocess import call #调用call函数使用cmd
from nilearn.signal import clean
from joblib import Parallel, delayed
from tqdm import tqdm
import time

In [24]:
#——————————————————————设置文件路径————————————————————
cifti_dir = ["/mnt/f/HCP1200_1","/mnt/g/HCP1200_2"] #HCP cifti功能像数据的存放路径, 由于空间不足分别存在两个盘
nifti_dir = "/mnt/h/HCP1200/REST2_preproc"
structure_dir="/mnt/h/HCP1200/Structure" #结构像数据存放的路径

session = ["rfMRI_REST1_LR", "rfMRI_REST2_LR"][1] #待分析的任务, 逐一分析

subjects = np.loadtxt("/mnt/e/liuwei/results/HCP1200fixsub.txt",dtype = str)#之前分析的所有被试编号,工作站路径
#subjects = np.loadtxt("/mnt/d/LWfiles/liuwei/HCP1200sub.txt",dtype = str)

ribbon_path = np.array([op.join(structure_dir,subjects[i],"MNINonLinear/ribbon.nii.gz") for i in range(len(subjects))]) #1018个被试的ribbon文件路径
print(type(ribbon_path))
checkRibbon = np.array([op.exists(i) for i in ribbon_path])#检查文件路径是否正确
ribbon_path = ribbon_path[checkRibbon]

wmparc_path = np.array([op.join(structure_dir,subjects[i],"MNINonLinear/wmparc.nii.gz") for i in range(len(subjects))]) #1018个被试的ribbon文件路径
checkWmparc = np.array([op.exists(i) for i in wmparc_path])#检查文件路径是否正确
wmparc_path = wmparc_path[checkWmparc]

cifti_path = np.array(["{0}/{1}_3T_rfMRI_REST_fix/{1}/MNINonLinear/Results/{2}/{2}_Atlas_MSMAll_hp2000_clean.dtseries.nii"
                 .format(i,j,session) for i in cifti_dir for j in subjects])#待分析的cifti文件路径
checkC = np.array([op.exists(i) for i in cifti_path])#检查文件路径是否正确
cifti_path = cifti_path[checkC]

if session == "rfMRI_REST2_LR" :
    nifti_path = np.array(["{0}/{1}_3T_rfMRI_REST2_preproc/{1}/MNINonLinear/Results/{2}/{2}.nii.gz"
                     .format(nifti_dir,j,session) for j in subjects])
    motionPar_path = np.array(["{0}/{1}_3T_rfMRI_REST2_preproc/{1}/MNINonLinear/Results/{2}/Movement_Regressors_dt.txt"
                 .format(nifti_dir,j,session) for j in subjects])
elif session ==  "rfMRI_REST1_LR":
     nifti_path = np.array(["/mnt/i/{0}/{1}/{0}.nii.gz".format(session,j) for j in subjects])
     motionPar_path = np.array(["/mnt/i/{0}/{1}/Movement_Regressors_dt.txt".format(session,j) for j in subjects])
    
checkN = np.array([op.exists(i) for i in nifti_path])
nifti_path = nifti_path[checkN]
checkMotion = np.array([op.exists(i) for i in motionPar_path])
motionPar_path = motionPar_path[checkMotion]

out_Dir = op.join("/mnt/e/HCP1200denoised/intercept",session)#设置输出文件夹，可以根据自己方便新建一个

<class 'numpy.ndarray'>


In [11]:
def pipeline(ribbon_path, wmparc_path, nifti_path, cifti_path, motionPar_path,#所需文件的路径
             subNum, outDir,session, high_pass, low_pass):
   #start = time.time()
    out_Dir = op.join(outDir, subNum)
    os.makedirs(out_Dir, exist_ok=True)
    maskAll, maskWM, maskCSF, maskGM = maskConstruction(ribbon_path, wmparc_path, nifti_path, out_Dir)
    V_demean = demean(nifti_path, maskAll, False) #对nifti文件标准化
    C_demean = demean(cifti_path, maskAll, True) #对cifti文件标准化
    V_dt = detrending(V_demean) #对nifti文件去趋势
    C_dt = detrending(C_demean) #对cifti文件去趋势
    C_denoising = signalRegress(motionPar_path, maskWM, maskCSF, maskGM, dt_Vdata = V_dt, dt_Cdata = C_dt, Rmotion=False, RGS = True, iscifti = True)
    TR = nib.load(nifti_path).header.structarr['pixdim'][4]
    data_flitered = flitering(C_denoising, TR, high=high_pass, low=low_pass)
    raw_cifti = nib.load(cifti_path)
    denoised_cifti = nib.Cifti2Image(data_flitered, #数据文件，时间×顶点二维数组
                                header = raw_cifti.header, #cifti的头文件
                                nifti_header=raw_cifti.nifti_header)
    #将文件保存到本地
    nib.save(denoised_cifti, op.join(out_Dir, "{}_denoised_{}.dtseries.nii".format(subNum, session)))
    #end = time.time()
    #print("被试{}处理结束，剩余{}个被试，用时为{}秒".format(subjects[i], str(len(ribbon_path)-i-1), str(end - start)))

In [ ]:
subNum = subjects[0]
out_Dir = op.join(out_Dir, subNum)
maskAll, maskWM, maskCSF, maskGM = maskConstruction(ribbon_path[0], wmparc_path[0], nifti_path[0], out_Dir)

In [ ]:
V_demean = demean(nifti_path[0], maskAll, False) #对nifti文件标准化
C_demean = demean(cifti_path[0], maskAll, True) #对cifti文件标准化
V_dt = detrending(V_demean) #对nifti文件去趋势
C_dt = detrending(C_demean) #对cifti文件去趋势

In [ ]:
for i in range(len(ribbon_path)):
    if i < 134:
        continue
    start = time.time()
    try:
        pipeline(ribbon_path[i], wmparc_path[i], nifti_path[i], cifti_path[i], motionPar_path[i],
                                   subjects[i], out_Dir,session, 0.01, 0.08)
    except:
        print("{}数据出现异常".format(subjects[i]))
    end = time.time()
    print("被试{}处理结束，剩余{}个被试，用时为{}秒".format(subjects[i], str(len(ribbon_path)-i-1), str(end - start)))